In [1]:
#%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import random
import os
import time
from utils import *

In [3]:
#!pip install ipympl

In [4]:
print("Pytorch version: ... {} ".format(torch.__version__))
print("Python version ... {}".format(sys.version))

Pytorch version: ... 1.12.1 
Python version ... 3.9.16 (main, Mar  8 2023, 14:00:05) 
[GCC 11.2.0]


In [5]:
# compute epoch time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [8]:
def get_random_DH():
    
    # nDoF: number of degrees of freedom
    max_nDoF = 6
    link_length_range = (100/1000, 1000/1000)
    
    
    nDoF = torch.randint(2, max_nDoF, (1,1))
    rDH = torch.zeros(nDoF, 4)
    
    # rDH[:,0]: theta
    rDH[:,0] = torch.rand(nDoF)
    #qlim = torch.zeros(nDoF,2)
    #qlim[:,0] = -torch.pi 
    #qlim[:,1] = torch.pi 
    #rDH[:,0] = torch.distributions.uniform.Uniform(qlim[:,0], qlim[:,1]).sample()
    
    # rDH[:,1]:
    rDH[:,1] = torch.empty(nDoF, dtype=torch.float32).uniform_(link_length_range[0], link_length_range[1])
    
    # rDH[:,2]: a
    rDH[:,2] = torch.empty(nDoF, dtype=torch.float32).uniform_(link_length_range[0], link_length_range[1]) 
    
    # rDH[:,3]: alpha
    alpha_choice = [0, torch.pi,torch.pi/2,torch.pi/3]
    alpha_signs = [-1, 1]
    rDH[:,3] = torch.tensor(
                            [random.choice(alpha_choice) * random.choice(alpha_signs) for _ in range(nDoF)],
                            dtype=torch.float32
                            )
    
    T = forward_kinematics(rDH)

    if not is_orthogonal(T):
        print(f"Warning: Transformation matrix for joint {i+1} is not orthogonal.")

    
    return rDH, nDoF
    

    
# Function to check orthogonality of rotation matrix
def is_orthogonal(matrix):
    """Check if a rotation matrix is orthogonal by verifying if R * R^T = I."""
    rotation_part = matrix[:3, :3]  # Extract rotation part
    return np.allclose(np.dot(rotation_part, rotation_part.T), np.eye(3), atol=1e-6)


In [11]:
rDH, nDoF = get_random_DH()
print("Generated {}DoF rDH =\n{}".format(nDoF.item(), rDH))

Valid: Transformation matrix for joint 1 is orthogonal.
Valid: Transformation matrix for joint 2 is orthogonal.
Valid: Transformation matrix for joint 3 is orthogonal.
Valid: Transformation matrix for joint 4 is orthogonal.
Generated 4DoF rDH =
tensor([[ 0.5487,  0.8594,  0.8946,  0.0000],
        [ 0.0619,  0.7459,  0.7006, -1.5708],
        [ 0.0682,  0.7484,  0.9375,  0.0000],
        [ 0.4747,  0.6320,  0.5575,  3.1416]])
